In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1459,2024-09-26,Mundo Amistosos Interclubes,14:00,Araski F (Esp),Gernika Bizkaia F (Esp),2.21,1.56,132.5,1.83,1.83,2.5,1.95,1.72,0foMOCP7,0.452489,0.641026,0.546448,0.546448,0.093514,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.243830,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1460,2024-09-26,Mundo Amistosos Interclubes,14:30,Uni Girona F (Esp),Estudiantes F (Esp),1.30,3.07,132.5,1.83,1.83,-8.5,1.94,1.73,O8vsmUeC,0.769231,0.325733,0.546448,0.546448,0.094964,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.572805,0.000000,0.080922,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1461,2024-09-26,Noruega Blno,14:00,Bærum,Kongsberg Miners,3.52,1.24,159.5,1.82,1.84,7.5,2.02,1.67,KnzGoRE4,0.284091,0.806452,0.549451,0.543478,0.090543,472.218,204.485124,0.433031,1.2,1.643168,1.369306,694.66,5.456,1.959638,0.359171,-31.0,107.118,16.586940,0.154847,1.2,1.643168,1.369306,89.60,1.232,0.129692,0.105269,3.0,94,70,7.39,1.28,197.038,110.216,0.677396,0.007728,0.134139,1.05,0.210,12.000000,0.599782,0.6,0.000218,-0.31,-0.062,-3.870968,0.849112,0.9,0.050888
1462,2024-09-26,Paraguai Lnb,20:30,Libertad,Felix Perez Cardozo,1.95,1.72,149.5,1.83,1.83,1.5,1.83,1.83,YLMtqfjn,0.512821,0.581395,0.546448,0.546448,0.094216,224.820,197.598786,0.878920,1.2,1.643168,1.369306,65.52,3.104,2.766935,0.891410,-23.0,201.690,197.885412,0.981136,1.8,1.643168,0.912871,94.86,2.996,3.578859,1.194546,31.0,63,93,1.04,1.02,223.524,382.114,0.088629,0.000000,0.000000,-2.89,-0.578,-1.643599,0.000000,0.0,0.000000,-3.74,-0.748,-0.962567,0.000000,0.0,0.000000
1463,2024-09-26,Rússia Liga Vtb United,13:00,Unics Kazan,Samara,1.11,6.75,154.5,1.82,1.92,-14.5,2.03,1.66,djpQS9sB,0.900901,0.148148,0.549451,0.520833,0.049049,175.476,52.200347,0.297479,1.8,1.643168,0.912871,260.16,2.022,0.572949,0.283357,15.0,294.354,190.653238,0.647701,0.6,1.341641,2.236068,139.04,4.194,2.915558,0.695174,-51.0,96,79,2.71,1.76,129.076,295.734,1.014779,0.037813,0.141805,-0.42,-0.084,-1.309524,0.710140,0.7,-0.010140,-2.96,-0.592,-9.712838,0.461535,0.5,0.038465
1464,2024-09-26,Europa Copa Europeia Da Fiba,14:30,Dnipro,CSM Constanta,5.66,1.12,153.5,1.82,1.84,10.5,1.95,1.76,UHNHwFkn,0.176678,0.892857,0.549451,0.543478,0.069536,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.946981,0.007728,0.072426,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1465,2024-09-26,Vietnã Vba,09:30,Can Tho Catfish,Saigon Heat,2.50,1.51,162.5,1.83,1.83,3.5,1.91,1.77,AJHpabcf,0.400000,0.662252,0.546448,0.546448,0.062252,188.812,46.930840,0.248559,1.2,1.643168,1.369306,240.30,2.320,0.549409,0.236814,-22.0,131.408,10.349846,0.078761,2.4,1.341641,0.559017,135.24,1.462,0.163156,0.111598,67.0,89,98,2.70,1.38,0.000,0.000,0.349145,0.000000,0.053802,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1466,2

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
